In [1]:
from pyspark.sql import SparkSession

In [2]:
# Step 1: Create a Spark session
spark = SparkSession.builder \
    .appName("Netflix Dataset Analysis") \
    .getOrCreate()

In [4]:
# Step 2: Load the CSV dataset
df = spark.read.csv("netflix_titles.csv", header=True, inferSchema=True)

In [5]:
# Step 3: Explore the data
df.printSchema()
df.show(5)
print(f"Total records: {df.count()}")

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+-------------------

In [8]:
from pyspark.sql.functions import col, sum, when


In [9]:
# Count nulls
df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).show()


+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   1|    2|    2636| 826|    832|        13|           2|     6|       5|        3|          3|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [19]:
# Fill some columns with default values
df_clean = df.fillna({
    "country": "Unknown",
    "director": "No Director",
    "cast": "No Cast",
    "rating": "Not Rated",
    "date_added": "Unknown",
    "duration": "Unknown",
    "listed_in": "Uncategorized",
    "description": "No description available"
})


In [20]:
# Check that nulls are gone
df_clean.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_clean.columns]).show()

+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   1|    2|       0|   0|      0|         0|           2|     0|       0|        0|          0|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [21]:
# Drop rows where release_year, title and typeis null
df_clean = df_clean.dropna(subset=["release_year","type","title"])

In [22]:
# Check that nulls are gone
df_clean.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_clean.columns]).show()

+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   0|    0|       0|   0|      0|         0|           0|     0|       0|        0|          0|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [23]:
# Step 4: Example Insights

In [34]:
# 1) Count of Movies vs TV Shows
df_clean.groupBy("type").count().show()


+-------+-----+
|   type|count|
+-------+-----+
|TV Show| 2676|
|  Movie| 6130|
+-------+-----+



In [35]:
# 2) Top 10 countries with most content
df_clean.groupBy("country").count().orderBy("count", ascending=False).show(10)


+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 2805|
|         India|  972|
|       Unknown|  830|
|United Kingdom|  419|
|         Japan|  245|
|   South Korea|  199|
|        Canada|  181|
|         Spain|  145|
|        France|  123|
|        Mexico|  110|
+--------------+-----+
only showing top 10 rows


In [26]:
# 3) Most common ratings
df_clean.groupBy("rating").count().orderBy("count", ascending=False).show()

+-----------------+-----+
|           rating|count|
+-----------------+-----+
|            TV-MA| 3195|
|            TV-14| 2158|
|            TV-PG|  862|
|                R|  796|
|            PG-13|  489|
|            TV-Y7|  334|
|             TV-Y|  307|
|               PG|  286|
|             TV-G|  220|
|               NR|   80|
|                G|   41|
|             NULL|    6|
|         TV-Y7-FV|    6|
|               UR|    3|
|            NC-17|    3|
|             2021|    2|
| November 1, 2020|    1|
| Shavidee Trotter|    1|
|    Adriane Lenox|    1|
|    Maury Chaykin|    1|
+-----------------+-----+
only showing top 20 rows


In [31]:
# 4) Recent releases: Shows added after 2020
from pyspark.sql.functions import col, to_date, year

df_new = df_clean.withColumn("date_added_clean", to_date(col("date_added"), "MMMM d, yyyy"))
df_new.filter(year(col("date_added_clean")) >= 2020).select("title", "type", "country", "date_added_clean").show(10)


+--------------------+-------+--------------------+----------------+
|               title|   type|             country|date_added_clean|
+--------------------+-------+--------------------+----------------+
|Dick Johnson Is Dead|  Movie|       United States|      2021-09-25|
|       Blood & Water|TV Show|        South Africa|      2021-09-24|
|           Ganglands|TV Show|             Unknown|      2021-09-24|
|Jailbirds New Orl...|TV Show|             Unknown|      2021-09-24|
|        Kota Factory|TV Show|               India|      2021-09-24|
|       Midnight Mass|TV Show|             Unknown|      2021-09-24|
|My Little Pony: A...|  Movie|             Unknown|      2021-09-24|
|             Sankofa|  Movie|United States, Gh...|      2021-09-24|
|The Great British...|TV Show|      United Kingdom|      2021-09-24|
|        The Starling|  Movie|       United States|      2021-09-24|
+--------------------+-------+--------------------+----------------+
only showing top 10 rows


In [32]:
# 5) Number of shows by year
df_clean.groupBy("release_year").count().orderBy("release_year").show(10)

+-----------------+-----+
|     release_year|count|
+-----------------+-----+
|   Charles Rocket|    1|
|          Dr. Dre|    1|
|   Francis Weddey|    1|
|     Imanol Arias|    1|
|      Jade Eshete|    1|
| Kristen Johnston|    1|
| Marquell Manning|    1|
|       Nick Kroll|    1|
|    Nse Ikpe-Etim|    1|
|       Paul Sambo|    1|
+-----------------+-----+
only showing top 10 rows


In [37]:
# 6) Average duration for movies
from pyspark.sql.functions import regexp_extract, when

# Keep only Movies with non-null, valid numeric durations
movies_df = df_clean.filter(col("type") == "Movie")

# Extract numbers safely using when
movies_df = movies_df.withColumn(
    "duration_mins",
    when(
        regexp_extract("duration", r'(\d+)', 1) != "",
        regexp_extract("duration", r'(\d+)', 1).cast("int")
    ).otherwise(None)
)

# Verify: see some rows
movies_df.select("duration", "duration_mins").show(10)

# Now safely compute the average
movies_df.selectExpr("avg(duration_mins) as avg_movie_duration").show()



+--------+-------------+
|duration|duration_mins|
+--------+-------------+
|  90 min|           90|
|  91 min|           91|
| 125 min|          125|
| 104 min|          104|
| 127 min|          127|
|  91 min|           91|
|  67 min|           67|
|  94 min|           94|
| 161 min|          161|
|  61 min|           61|
+--------+-------------+
only showing top 10 rows
+------------------+
|avg_movie_duration|
+------------------+
|  99.8750204482251|
+------------------+

